In [1]:
import torch
import torch.nn as nn

In [2]:
# Source for char embeddings - https://github.com/minimaxir/char-embeddings/tree/master/output
embedding_path='char-embeddings.txt'

In [3]:
import torchtext.vocab as vocab

custom_embeddings = vocab.Vectors(name = embedding_path)

In [4]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [5]:
def get_mean_word_embedding(word):
    wordembed=custom_embeddings.get_vecs_by_tokens([*word])
    # take the mean
    wordembed=wordembed.mean(dim=0)
    return wordembed

In [6]:
def getSimilarity(str1,str2):
    str1embed=get_mean_word_embedding(str1)
    str2embed=get_mean_word_embedding(str2)
    return cos(str1embed, str2embed)

In [7]:
getSimilarity('Abhijit Shingote','ShiotAbhijit')

tensor(0.9883)

In [8]:
def getlines(path,line_numbers=[]):
    with open(path, 'r') as fp:
        lines = []
        for i, line in enumerate(fp):
            if i in line_numbers:
                lines.append((i,line))
    return lines

In [9]:
def getSimilarityScores(query,path,n=5):
    similarity_scores=[]
    with open(path) as f:
        for line in f:
            similarity_scores.append(getSimilarity(query,line))
    top_indexes=sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i])[-n:]
    top_scores=[(i,similarity_scores[i]) for i in top_indexes]
    top_scores=sorted(top_scores,key=lambda i:i[0])
    lines=getlines(path,top_indexes)
    lines=sorted(lines,key=lambda i:i[0])
    final=list()
    final=[(score[1].item(),line[1]) for score,line in zip(top_scores,lines)]
    
    return sorted(final,reverse=True)

In [10]:
getSimilarityScores('2022-11-16_','universe.txt')

[(0.9473178386688232,
  '2022-11-16_000000328_in-network-rates_10_of_25.json.gz\n'),
 (0.9465864896774292,
  '2022-11-16_000000328_in-network-rates_02_of_25.json.gz\n'),
 (0.9459896683692932,
  '2022-11-16_000002133_in-network-rates_05_of_07.json.gz\n'),
 (0.9449851512908936,
  '2022-11-16_000000106_in-network-rates_06_of_15.json.gz\n'),
 (0.9446848630905151,
  '2022-11-16_000000328_in-network-rates_04_of_25.json.gz\n')]